# Tabla de resultados
1. Lectura y todos los subresultados

In [1]:
## Get current work directory to load all the custom modules
from pathlib import Path
import sys
BASE_DIR = Path().absolute().resolve(strict=True).parent
if str(BASE_DIR) not in sys.path:
    sys.path.append(str(BASE_DIR))

In [2]:
DATA_PATH = os.path.join(BASE_DIR, 'data', 'autos')
print(DATA_PATH)

/workspace/Externos/TimeSeries/BenchMark/data/autos


In [3]:
import datetime as dt
import pandas as pd
import glob

># 1. Lectura y Formateo de datos

In [4]:
csv_paths1 = glob.glob(os.path.join(DATA_PATH + "/result/**/", '*.csv'))
csv_paths2 = glob.glob(os.path.join(DATA_PATH + "/result/**/**/", '*.csv'))
csv_paths = csv_paths1 + csv_paths2

### MSE - tabla de resultados

In [5]:
# Hyperparameters by model
for i, csv_result in enumerate(csv_paths):
    result = pd.read_csv(csv_result, usecols=['idArticulo', 'hyper', 'model'])
    model_name = result.model.sample().tolist()[0]
    result.drop(['model'], axis=1, inplace=True)
    result.columns = ['idArticulo', model_name]
    if i == 0:
        data_hyper = result
    else:
        data_hyper = pd.merge(data_hyper, result, on='idArticulo', how='outer')
data_hyper = data_hyper.fillna(1e2)
data_hyper = data_hyper.set_index('idArticulo')
data_hyper.head()

,ARIMA,DL,LR,DT,XGB
idArticulo,,,,,
551.0,"{'ar': 2, 'ii': 1, 'ma': 2}","{'n_lags': 6, 'random_state': 2073, 'optimizer...",{'n_lags': 6},"{'n_lags': 2, 'max_depth': 7, 'random_state': ...","{'n_lags': 2, 'max_depth': 6, 'random_state': ..."
594.0,"{'ar': 2, 'ii': 3, 'ma': 5}","{'n_lags': 6, 'random_state': 1474, 'optimizer...",{'n_lags': 5},"{'n_lags': 7, 'max_depth': 3, 'random_state': ...","{'n_lags': 3, 'max_depth': 4, 'random_state': ..."
963.0,"{'ar': 4, 'ii': 1, 'ma': 0}","{'n_lags': 4, 'random_state': 1944, 'optimizer...",{'n_lags': 2},"{'n_lags': 4, 'max_depth': 7, 'random_state': ...","{'n_lags': 4, 'max_depth': 3, 'random_state': ..."
1208.0,"{'ar': 2, 'ii': 1, 'ma': 0}","{'n_lags': 7, 'random_state': 2340, 'optimizer...",{'n_lags': 11},"{'n_lags': 5, 'max_depth': 5, 'random_state': ...","{'n_lags': 6, 'max_depth': 3, 'random_state': ..."
1224.0,"{'ar': 4, 'ii': 2, 'ma': 6}","{'n_lags': 6, 'random_state': 1083, 'optimizer...",{'n_lags': 2},"{'n_lags': 5, 'max_depth': 5, 'random_state': ...","{'n_lags': 3, 'max_depth': 5, 'random_state': ..."


In [6]:
# MSE by model
for i, csv_result in enumerate(csv_paths):
    result = pd.read_csv(csv_result, usecols=['idArticulo', 'mse_test', 'model'])
    model_name = result.model.sample().tolist()[0]
    result.drop(['model'], axis=1, inplace=True)
    result.columns = ['idArticulo', model_name]
    if i == 0:
        data = result
    else:
        data = pd.merge(data, result, on='idArticulo', how='outer')
data = data.fillna(1e2)
data.head()

,idArticulo,ARIMA,DL,LR,DT,XGB
0,551.0,0.404937,0.421291,0.360980,0.502383,0.095949
1,594.0,3.840954,3.279004,3.796934,0.876083,1.999576
2,963.0,1.996980,1.027604,1.586770,2.366010,1.327967
3,1208.0,0.089191,0.032514,0.239862,0.163267,0.114811
4,1224.0,1.727660,1.356769,1.295823,2.211029,0.673324


In [7]:
data = data.set_index('idArticulo')
columns = data.columns.tolist()
min_mse = data.min(axis=1).tolist()

def set_model(x, columns, hyper):
    index = x.tolist().index(x.min())
    hyper = hyper.loc[x.name, columns[index]]
    return pd.Series(index=['model', 'hyper'], data=[columns[index], hyper])
print("="*20 + " RESUMEN MSE " + "="*20)
hyper_model = data.apply(set_model, args=(columns, data_hyper ), axis=1)
data = data.join(hyper_model)
data.to_csv(os.path.join(DATA_PATH, 'resumen_mse.csv'))
data
data.style.background_gradient(cmap='Blues')


==================== RESUMEN MSE ====================


,ARIMA,DL,LR,DT,XGB,model,hyper
idArticulo,,,,,,,
551.0,0.404937,0.421291,0.360980,0.502383,0.095949,XGB,"{'n_lags': 2, 'max_depth': 6, 'random_state': 410, 'gamma': 0.1989850279800391, 'n_estimators': 10}"
594.0,3.840954,3.279004,3.796934,0.876083,1.999576,DT,"{'n_lags': 7, 'max_depth': 3, 'random_state': 973, 'ccp_alpha': 0.09567362522116188}"
963.0,1.996980,1.027604,1.586770,2.366010,1.327967,DL,"{'n_lags': 4, 'random_state': 1944, 'optimizer': 'SGD', 'learning_rate': 0.0001, 'n_epochs': 150}"
1208.0,0.089191,0.032514,0.239862,0.163267,0.114811,DL,"{'n_lags': 7, 'random_state': 2340, 'optimizer': 'SGD', 'learning_rate': 7.000000000000001e-05, 'n_epochs': 150}"
1224.0,1.727660,1.356769,1.295823,2.211029,0.673324,XGB,"{'n_lags': 3, 'max_depth': 5, 'random_state': 2718, 'gamma': 0.6894119958309484, 'n_estimators': 9}"
1627.0,1.186640,2.584031,1.455596,5.720230,1.046388,XGB,"{'n_lags': 4, 'max_depth': 3, 'random_state': 1904, 'gamma': 0.13055965414451284, 'n_estimators': 2}"
3090.0,2.413196,1.565380,0.757318,3.223051,1.890009,LR,{'n_lags': 2}
3882.0,0.037977,1.350949,0.166846,0.860024,0.552605,ARIMA,"{'ar': 4, 'ii': 2, 'ma': 5}"
3883.0,1.534794,2.128066,1.180694,1.827396,0.960361,XGB,"{'n_lags': 5, 'max_depth': 5, 'random_state': 1230, 'gamma': 0.0056306132846087875, 'n_estimators': 2}"
